# 🍽️ 네이버 플레이스 레스토랑 크롤링 (정리본)

## 📋 기능
- 네이버 지도에서 레스토랑 검색
- 다중 페이지 네비게이션 지원
- **📜 페이지별 스크롤 기능으로 모든 레스토랑 로드**
- **🆔 Place ID 자동 추출 및 URL 생성**
- 레스토랑 상세정보 + 메뉴 크롤링
- "펼쳐서 더보기" 자동 클릭

## ⚡ 사용법
1. 라이브러리 import
2. Chrome 브라우저 설정
3. 크롤링 실행 (스크롤 옵션 포함)

## 1️⃣ 라이브러리 Import

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time
import re
from datetime import datetime

print("📦 라이브러리 import 완료")

📦 라이브러리 import 완료


## 2️⃣ Chrome 브라우저 설정

In [2]:
def setup_chrome_driver():
    """Chrome 브라우저 설정 및 초기화"""
    print("🔧 Chrome 브라우저 설정 중...")
    
    options = webdriver.ChromeOptions()
    options.add_experimental_option("detach", True)
    options.add_argument("--window-size=1400,1000")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)
    
    try:
        driver = webdriver.Chrome(
            service=Service(ChromeDriverManager().install()), 
            options=options
        )
        driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
        print("✅ Chrome 브라우저 초기화 완료!")
        return driver
    except Exception as e:
        print(f"❌ 브라우저 초기화 실패: {e}")
        return None

# 브라우저 초기화
driver = setup_chrome_driver()
if driver:
    print("🎉 브라우저 설정 완료!")
else:
    print("💥 브라우저 설정 실패")

🔧 Chrome 브라우저 설정 중...


✅ Chrome 브라우저 초기화 완료!
🎉 브라우저 설정 완료!


## 3️⃣ 핵심 크롤링 함수들

In [3]:
def search_naver_map(driver, search_keyword):
    """네이버 지도에서 검색"""
    try:
        map_url = f"https://map.naver.com/p/search/{search_keyword}"
        print(f"🌐 네이버 지도 접속: {search_keyword}")
        
        driver.get(map_url)
        time.sleep(5)
        
        # searchIframe으로 전환
        driver.switch_to.frame("searchIframe")
        time.sleep(3)
        
        # 검색 결과 확인
        results = driver.find_elements(By.CSS_SELECTOR, ".place_bluelink")
        if results:
            print(f"✅ 검색 성공: {len(results)}개 결과 발견")
            return True
        else:
            print("❌ 검색 결과 없음")
            return False
            
    except Exception as e:
        print(f"❌ 검색 실패: {e}")
        return False

In [4]:
def crawl_single_restaurant(driver, restaurant_index=0):
    """단일 레스토랑 상세정보 크롤링 (Place ID 포함)"""
    print(f"🏪 {restaurant_index + 1}번째 가게 크롤링 시작")
    
    restaurant_data = {
        'index': restaurant_index,
        'name': '',
        'category': '',
        'address': '',
        'contact': '',
        'place_id': '',
        'place_url': '',
        'menus': [],
        'crawled_at': '',
        'success': False
    }
    
    try:
        # TYaxT 버튼 찾기
        tyaxt_buttons = driver.find_elements(By.CSS_SELECTOR, "span.TYaxT")
        
        if len(tyaxt_buttons) <= restaurant_index:
            print(f"❌ {restaurant_index + 1}번째 가게 없음")
            return restaurant_data
        
        target_button = tyaxt_buttons[restaurant_index]
        restaurant_name = target_button.text.strip()
        print(f"🎯 타겟: {restaurant_name}")
        
        # Place ID 추출 (클릭하기 전에)
        try:
            # li 요소에서 place_bluelink 찾기
            li_element = target_button.find_element(By.XPATH, "./ancestor::li")
            place_link = li_element.find_element(By.CSS_SELECTOR, "a.place_bluelink")
            
            # place_bluelink 클릭하여 Place ID 추출
            driver.execute_script("arguments[0].click();", place_link)
            time.sleep(1.5)
            
            # URL에서 Place ID 추출
            current_url = driver.current_url
            place_id_match = re.search(r'/place/(\d+)', current_url)
            
            if place_id_match:
                place_id = place_id_match.group(1)
                restaurant_data['place_id'] = place_id
                restaurant_data['place_url'] = f"https://map.naver.com/p/entry/place/{place_id}"
                print(f"🆔 Place ID: {place_id}")
            else:
                print("⚠️ Place ID 추출 실패")
            
            # searchIframe으로 돌아가기
            driver.switch_to.default_content()
            driver.switch_to.frame("searchIframe")
            time.sleep(1)
            
        except Exception as e:
            print(f"⚠️ Place ID 추출 중 오류: {e}")
        
        # TYaxT 버튼 다시 찾아서 클릭 (상세 페이지로 이동)
        tyaxt_buttons = driver.find_elements(By.CSS_SELECTOR, "span.TYaxT")
        if len(tyaxt_buttons) > restaurant_index:
            target_button = tyaxt_buttons[restaurant_index]
            driver.execute_script("arguments[0].click();", target_button)
            time.sleep(3)
        
        # entryIframe으로 전환
        driver.switch_to.default_content()
        driver.switch_to.frame("entryIframe")
        time.sleep(3)
        
        # 기본 정보 수집
        try:
            name_elem = driver.find_element(By.CSS_SELECTOR, "span.GHAhO")
            restaurant_data['name'] = name_elem.text.strip()
        except:
            restaurant_data['name'] = restaurant_name
        
        try:
            category_elem = driver.find_element(By.CSS_SELECTOR, "span.lnJFt")
            restaurant_data['category'] = category_elem.text.strip()
        except:
            pass
            
        try:
            address_elem = driver.find_element(By.CSS_SELECTOR, "span.LDgIH")
            restaurant_data['address'] = address_elem.text.strip()
        except:
            pass
            
        try:
            contact_elem = driver.find_element(By.CSS_SELECTOR, "span.xlx7Q")
            restaurant_data['contact'] = contact_elem.text.strip()
        except:
            pass
        
        print(f"📛 {restaurant_data['name']} ({restaurant_data['category']})")
        
        # 메뉴 탭 클릭 및 메뉴 수집
        try:
            tab_container = driver.find_element(By.CSS_SELECTOR, ".place_fixed_maintab")
            tabs = tab_container.find_elements(By.TAG_NAME, "a")
            
            menu_tab_found = False
            for tab in tabs:
                if '메뉴' in tab.text:
                    driver.execute_script("arguments[0].click();", tab)
                    time.sleep(3)
                    menu_tab_found = True
                    break
            
            if menu_tab_found:
                # "펼쳐서 더보기" 버튼 클릭
                expand_attempts = 0
                while expand_attempts < 5:
                    try:
                        expand_spans = driver.find_elements(By.XPATH, 
                            "//span[@class='TeItc' and contains(text(), '펼쳐서 더보기')]")
                        
                        if not expand_spans:
                            break
                            
                        for span in expand_spans:
                            parent = span.find_element(By.XPATH, "..")
                            if parent.tag_name == 'a' and 'fvwqf' in parent.get_attribute("class"):
                                driver.execute_script("arguments[0].click();", parent)
                                time.sleep(2)
                                expand_attempts += 1
                                break
                        else:
                            break
                    except:
                        break
                
                # 메뉴 리스트 수집
                try:
                    menu_items = driver.find_elements(By.CSS_SELECTOR, "li.E2jtL")
                    print(f"🍽️ {len(menu_items)}개 메뉴 발견")
                    
                    for menu_li in menu_items:
                        try:
                            name_elem = menu_li.find_element(By.CSS_SELECTOR, ".lPzHi")
                            menu_name = name_elem.text.strip()
                            
                            price_num = None
                            price_text = ""
                            try:
                                price_elem = menu_li.find_element(By.CSS_SELECTOR, ".GXS1X em")
                                price_text = price_elem.text.strip()
                                price_clean = re.sub(r'[^\d]', '', price_text)
                                if price_clean:
                                    price_num = int(price_clean)
                            except:
                                pass
                            
                            if menu_name:
                                restaurant_data['menus'].append({
                                    'name': menu_name,
                                    'price': price_num,
                                    'price_text': price_text
                                })
                        except:
                            continue
                    
                    print(f"✅ {len(restaurant_data['menus'])}개 메뉴 수집 완료")
                except Exception as e:
                    print(f"⚠️ 메뉴 수집 실패: {e}")
        except Exception as e:
            print(f"⚠️ 메뉴 탭 처리 실패: {e}")
        
        restaurant_data['crawled_at'] = datetime.now().isoformat()
        restaurant_data['success'] = True
        print(f"✅ 크롤링 완료!")
        
    except Exception as e:
        print(f"❌ 크롤링 실패: {e}")
    
    finally:
        # searchIframe으로 돌아가기
        try:
            driver.switch_to.default_content()
            driver.switch_to.frame("searchIframe")
            time.sleep(1)
        except:
            pass
    
    return restaurant_data

In [5]:
def scroll_and_load_all_restaurants(driver, max_scroll_attempts=10):
    """페이지에서 스크롤하여 모든 레스토랑 로드"""
    print("📜 스크롤하여 모든 레스토랑 로드 중...")
    
    initial_count = 0
    stable_count = 0
    scroll_attempts = 0
    
    try:
        # 초기 레스토랑 수 확인
        tyaxt_buttons = driver.find_elements(By.CSS_SELECTOR, "span.TYaxT")
        initial_count = len(tyaxt_buttons)
        print(f"📋 초기 레스토랑 수: {initial_count}개")
        
        while scroll_attempts < max_scroll_attempts:
            # 스크롤 컨테이너 찾기
            try:
                scroll_container = driver.find_element(By.CSS_SELECTOR, "#_pcmap_list_scroll_container")
            except:
                try:
                    scroll_container = driver.find_element(By.CSS_SELECTOR, ".Ryr1F")
                except:
                    print("⚠️ 스크롤 컨테이너를 찾을 수 없음")
                    break
            
            # 현재 레스토랑 수 확인
            current_tyaxt_buttons = driver.find_elements(By.CSS_SELECTOR, "span.TYaxT")
            current_count = len(current_tyaxt_buttons)
            
            # 스크롤 다운
            driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scroll_container)
            time.sleep(2)
            
            # 스크롤 후 레스토랑 수 재확인
            new_tyaxt_buttons = driver.find_elements(By.CSS_SELECTOR, "span.TYaxT")
            new_count = len(new_tyaxt_buttons)
            
            print(f"📜 스크롤 {scroll_attempts + 1}: {current_count} → {new_count}개")
            
            # 변화가 없으면 안정화 카운트 증가
            if new_count == current_count:
                stable_count += 1
                if stable_count >= 3:  # 3번 연속 변화 없으면 종료
                    print("✅ 모든 레스토랑 로드 완료 (더 이상 추가되지 않음)")
                    break
            else:
                stable_count = 0  # 변화가 있으면 안정화 카운트 리셋
            
            scroll_attempts += 1
        
        # 최종 레스토랑 수 확인
        final_tyaxt_buttons = driver.find_elements(By.CSS_SELECTOR, "span.TYaxT")
        final_count = len(final_tyaxt_buttons)
        
        print(f"📊 스크롤 완료: {initial_count} → {final_count}개 ({final_count - initial_count}개 추가)")
        return final_count
        
    except Exception as e:
        print(f"❌ 스크롤 중 오류: {e}")
        # 오류 발생 시 현재 레스토랑 수 반환
        try:
            fallback_buttons = driver.find_elements(By.CSS_SELECTOR, "span.TYaxT")
            return len(fallback_buttons)
        except:
            return initial_count

In [6]:
def crawl_multiple_pages(driver, search_keyword, max_pages=None, max_restaurants_per_page=None, enable_scroll=True):
    """다중 페이지 레스토랑 크롤링 (메인 함수)"""
    print(f"🚀 다중 페이지 크롤링 시작")
    print(f"🔍 검색어: '{search_keyword}'")
    if max_pages:
        print(f"📄 최대 페이지: {max_pages}")
    else:
        print(f"📄 페이지 제한: 무제한 (모든 페이지)")
    print(f"📜 스크롤 활성화: {enable_scroll}")
    if max_restaurants_per_page:
        print(f"🏪 페이지당 최대 레스토랑: {max_restaurants_per_page}개")
    else:
        print(f"🏪 페이지당 레스토랑: 무제한 (모든 레스토랑)")
    print("=" * 60)
    
    # 검색 시작
    if not search_naver_map(driver, search_keyword):
        return {"success": False, "restaurants": [], "message": "검색 실패"}
    
    all_restaurants = []
    page_counts = {}
    current_page = 1
    max_failed_attempts = 3  # 연속 실패 시 종료
    failed_attempts = 0
    
    while True:  # 무제한 루프 (조건에 따라 break)
        # 페이지 제한 확인
        if max_pages and current_page > max_pages:
            print(f"📄 최대 페이지({max_pages}) 도달로 종료")
            break
            
        print(f"\n📄 페이지 {current_page} 크롤링...")
        
        try:
            # searchIframe 확인
            driver.switch_to.default_content()
            driver.switch_to.frame("searchIframe")
            time.sleep(2)
            
            # 스크롤하여 모든 레스토랑 로드 (옵션)
            if enable_scroll:
                total_loaded = scroll_and_load_all_restaurants(driver, max_scroll_attempts=10)
            
            # 현재 페이지 레스토랑 수 확인
            tyaxt_buttons = driver.find_elements(By.CSS_SELECTOR, "span.TYaxT")
            available_restaurants = len(tyaxt_buttons)
            
            # 크롤링할 레스토랑 수 결정
            if max_restaurants_per_page is None:
                page_restaurant_count = available_restaurants  # 모든 레스토랑
                print(f"📋 페이지의 모든 레스토랑 크롤링: {available_restaurants}개")
            else:
                page_restaurant_count = min(available_restaurants, max_restaurants_per_page)
                if enable_scroll:
                    print(f"📋 스크롤 후 총 {available_restaurants}개 중 {page_restaurant_count}개 크롤링 예정")
                else:
                    print(f"📋 {page_restaurant_count}개 레스토랑 발견")
            
            if page_restaurant_count == 0:
                print("❌ 페이지에 레스토랑이 없음")
                failed_attempts += 1
                if failed_attempts >= max_failed_attempts:
                    print(f"🔚 연속 {max_failed_attempts}번 실패로 크롤링 종료")
                    break
                else:
                    print(f"⚠️ 다음 페이지 시도... ({failed_attempts}/{max_failed_attempts})")
                    current_page += 1
                    continue
            
            # 성공적으로 레스토랑 발견시 실패 카운트 리셋
            failed_attempts = 0
            
            # 각 레스토랑 크롤링
            page_restaurants = []
            success_count = 0
            
            for i in range(page_restaurant_count):
                print(f"\n🏪 {i+1}/{page_restaurant_count} 크롤링 중...")
                
                restaurant_data = crawl_single_restaurant(driver, i)
                
                if restaurant_data['success']:
                    restaurant_data['page_number'] = current_page
                    restaurant_data['position_in_page'] = i + 1
                    page_restaurants.append(restaurant_data)
                    success_count += 1
                    print(f"✅ 성공 (메뉴 {len(restaurant_data['menus'])}개)")
                else:
                    print(f"❌ 실패")
            
            all_restaurants.extend(page_restaurants)
            page_counts[f"page_{current_page}"] = success_count
            
            print(f"📊 페이지 {current_page} 완료: {success_count}개 성공")
            
            # 다음 페이지로 이동 시도
            print(f"\n🔄 다음 페이지로 이동...")
            
            try:
                pagination_container = driver.find_element(By.CSS_SELECTOR, ".zRM9F")
                next_buttons = pagination_container.find_elements(By.CSS_SELECTOR, "a.eUTV2")
                
                next_button = None
                for btn in next_buttons:
                    if "다음" in btn.text and btn.get_attribute("aria-disabled") != "true":
                        next_button = btn
                        break
                
                if next_button:
                    driver.execute_script("arguments[0].click();", next_button)
                    time.sleep(4)
                    current_page += 1
                    print(f"✅ 페이지 {current_page}로 이동 완료")
                else:
                    print("🔚 더 이상 다음 페이지 없음 - 모든 페이지 크롤링 완료!")
                    break
                    
            except Exception as e:
                print(f"❌ 페이지 이동 실패: {e}")
                print("🔚 페이지 네비게이션 종료")
                break
                
        except Exception as e:
            print(f"❌ 페이지 {current_page} 처리 실패: {e}")
            failed_attempts += 1
            if failed_attempts >= max_failed_attempts:
                print(f"🔚 연속 {max_failed_attempts}번 실패로 크롤링 종료")
                break
            else:
                current_page += 1
                continue
    
    # 최종 결과
    total_restaurants = len(all_restaurants)
    total_menus = sum(len(r["menus"]) for r in all_restaurants)
    
    print(f"\n🎉 크롤링 완료!")
    print("=" * 60)
    print(f"📊 총 레스토랑: {total_restaurants}개")
    print(f"🍽️ 총 메뉴: {total_menus}개")
    print(f"📄 처리된 페이지: {len(page_counts)}개")
    
    for page, count in page_counts.items():
        print(f"  - {page}: {count}개")
    
    return {
        "success": True,
        "restaurants": all_restaurants,
        "page_counts": page_counts,
        "total_restaurants": total_restaurants,
        "total_menus": total_menus
    }

## 4️⃣ 크롤링 실행

In [7]:
# 🚀 크롤링 실행
if 'driver' in locals() and driver:
    print("🎯 크롤링 시작!")
    
    # 크롤링 설정
    SEARCH_KEYWORD = "강남 맛집"    # 검색어 변경 가능
    MAX_PAGES = None              # 페이지 제한 없음 - 모든 페이지 크롤링
    MAX_RESTAURANTS_PER_PAGE = None # 제한 없음 - 페이지의 모든 레스토랑 크롤링
    ENABLE_SCROLL = True          # 스크롤 활성화 여부
    
    print(f"📊 설정:")
    print(f"  🔍 검색어: {SEARCH_KEYWORD}")
    print(f"  📄 페이지 제한: {'무제한 (모든 페이지)' if MAX_PAGES is None else f'{MAX_PAGES}페이지'}")
    print(f"  🏪 페이지당 레스토랑: {'무제한 (모든 레스토랑)' if MAX_RESTAURANTS_PER_PAGE is None else f'{MAX_RESTAURANTS_PER_PAGE}개'}")
    print(f"  📜 스크롤: {'활성화' if ENABLE_SCROLL else '비활성화'}")
    print()
    
    # 주의사항 표시
    if MAX_PAGES is None:
        print("⚠️  주의: 페이지 제한이 없어 많은 데이터가 수집될 수 있습니다.")
        print("   필요시 중간에 중단하거나 MAX_PAGES를 설정하세요.")
        print()
    
    # 실행
    result = crawl_multiple_pages(
        driver=driver,
        search_keyword=SEARCH_KEYWORD,
        max_pages=MAX_PAGES,
        max_restaurants_per_page=MAX_RESTAURANTS_PER_PAGE,
        enable_scroll=ENABLE_SCROLL
    )
    
    # 결과 확인
    if result["success"]:
        restaurants = result["restaurants"]
        print(f"\n🎉 크롤링 성공!")
        print(f"📊 수집된 레스토랑: {len(restaurants)}개")
        
        # 페이지별 통계
        if result.get("page_counts"):
            print(f"\n📄 페이지별 수집 현황:")
            for page, count in result["page_counts"].items():
                print(f"  {page}: {count}개")
        
        # 샘플 출력
        if restaurants:
            print(f"\n📋 샘플 레스토랑 (처음 3개):")
            for i, restaurant in enumerate(restaurants[:3], 1):
                print(f"  {i}. {restaurant['name']} ({restaurant['category']})")
                print(f"     📍 {restaurant['address'][:50]}...")
                print(f"     🍽️ 메뉴 {len(restaurant['menus'])}개")
                print(f"     📄 페이지 {restaurant.get('page_number', 'N/A')}")
                print()
    else:
        print("💥 크롤링 실패")
else:
    print("❌ 브라우저가 초기화되지 않았습니다. 위의 셀을 먼저 실행해주세요.")

🎯 크롤링 시작!
📊 설정:
  🔍 검색어: 강남 맛집
  📄 페이지 제한: 무제한 (모든 페이지)
  🏪 페이지당 레스토랑: 무제한 (모든 레스토랑)
  📜 스크롤: 활성화

⚠️  주의: 페이지 제한이 없어 많은 데이터가 수집될 수 있습니다.
   필요시 중간에 중단하거나 MAX_PAGES를 설정하세요.

🚀 다중 페이지 크롤링 시작
🔍 검색어: '강남 맛집'
📄 페이지 제한: 무제한 (모든 페이지)
📜 스크롤 활성화: True
🏪 페이지당 레스토랑: 무제한 (모든 레스토랑)
🌐 네이버 지도 접속: 강남 맛집
✅ 검색 성공: 10개 결과 발견

📄 페이지 1 크롤링...
📜 스크롤하여 모든 레스토랑 로드 중...
📋 초기 레스토랑 수: 10개
📜 스크롤 1: 10 → 10개
📜 스크롤 2: 10 → 10개
📜 스크롤 3: 10 → 10개
✅ 모든 레스토랑 로드 완료 (더 이상 추가되지 않음)
📊 스크롤 완료: 10 → 10개 (0개 추가)
📋 페이지의 모든 레스토랑 크롤링: 10개

🏪 1/10 크롤링 중...
🏪 1번째 가게 크롤링 시작
🎯 타겟: 복돈이 막회 물회 육회 해산물 역삼본점
⚠️ Place ID 추출 실패
❌ 크롤링 실패: Message: entryIframe

❌ 실패

🏪 2/10 크롤링 중...
🏪 2번째 가게 크롤링 시작
🎯 타겟: 참치한마리 역삼점
⚠️ Place ID 추출 실패
❌ 크롤링 실패: Message: entryIframe

❌ 실패

🏪 3/10 크롤링 중...
🏪 3번째 가게 크롤링 시작
🎯 타겟: 멕시타이거
🆔 Place ID: 1486617466
📛 멕시타이거 (요리주점)
🍽️ 41개 메뉴 발견
✅ 41개 메뉴 수집 완료
✅ 크롤링 완료!
✅ 성공 (메뉴 41개)

🏪 4/10 크롤링 중...
🏪 4번째 가게 크롤링 시작
🎯 타겟: 쿄코코 신논현점
🆔 Place ID: 1581701475
📛 쿄코코 신논현점 (일식당)
🍽️ 15개 메뉴 발견
✅ 15개 메뉴 수집 완료
✅ 크롤링 완료!
✅ 성공 (메뉴 15개)

🏪 

## 5️⃣ 결과 분석 및 저장

In [8]:
# 📊 결과 분석
if 'result' in locals() and result["success"]:
    restaurants = result["restaurants"]
    
    # 카테고리별 분석
    categories = {}
    for restaurant in restaurants:
        category = restaurant.get('category', '미분류')
        categories[category] = categories.get(category, 0) + 1
    
    print("📊 카테고리별 분포:")
    for category, count in sorted(categories.items(), key=lambda x: x[1], reverse=True):
        print(f"  {category}: {count}개")
    
    # 메뉴 통계
    total_menus = sum(len(r['menus']) for r in restaurants)
    avg_menus = total_menus / len(restaurants) if restaurants else 0
    
    print(f"\n🍽️ 메뉴 통계:")
    print(f"  총 메뉴 수: {total_menus}개")
    print(f"  평균 메뉴/레스토랑: {avg_menus:.1f}개")
    
    # 성공률
    success_count = sum(1 for r in restaurants if r.get('success', False))
    success_rate = success_count / len(restaurants) * 100 if restaurants else 0
    
    print(f"\n✅ 성공률: {success_count}/{len(restaurants)} ({success_rate:.1f}%)")
else:
    print("❌ 분석할 결과가 없습니다.")

📊 카테고리별 분포:
  육류,고기요리: 18개
  카페,디저트: 12개
  요리주점: 11개
  양식: 11개
  한식: 10개
  일식당: 8개
  중식당: 6개
  멕시코,남미음식: 4개
  초밥,롤: 4개
  이탈리아음식: 4개
  햄버거: 3개
  베이커리: 3개
  칼국수,만두: 2개
  일본식라면: 2개
  돈가스: 2개
  맥주,호프: 2개
  돼지고기구이: 2개
  냉면: 2개
  샤브샤브: 2개
  치킨,닭강정: 2개
  스테이크,립: 2개
  생선회: 2개
  이자카야: 2개
  닭갈비: 1개
  오리요리: 1개
  술집: 1개
  국수: 1개
  아이스크림: 1개
  정육식당: 1개
  : 1개
  장어,먹장어요리: 1개
  피자: 1개
  뷔페: 1개
  해물,생선요리: 1개
  아시아음식: 1개
  우동,소바: 1개
  순대,순댓국: 1개
  찌개,전골: 1개
  게요리: 1개
  족발,보쌈: 1개
  만두: 1개
  해산물뷔페: 1개
  닭요리: 1개
  태국음식: 1개
  종합분식: 1개
  곰탕,설렁탕: 1개
  닭발: 1개
  바(BAR): 1개
  닭볶음탕: 1개

🍽️ 메뉴 통계:
  총 메뉴 수: 3056개
  평균 메뉴/레스토랑: 21.5개

✅ 성공률: 142/142 (100.0%)


In [9]:
# 💾 결과 저장 (옵션)
if 'result' in locals() and result["success"]:
    import json
    
    # JSON 파일로 저장
    filename = f"naver_restaurants_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
    
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(result, f, ensure_ascii=False, indent=2)
    
    print(f"💾 결과 저장 완료: {filename}")
    print(f"📂 저장된 데이터: {len(result['restaurants'])}개 레스토랑")
else:
    print("❌ 저장할 결과가 없습니다.")

💾 결과 저장 완료: naver_restaurants_20250813_153837.json
📂 저장된 데이터: 142개 레스토랑


## 🔧 브라우저 종료

In [10]:
# 브라우저 종료 (필요시)
if 'driver' in locals() and driver:
    driver.quit()  # 주석 해제하면 브라우저 종료
    print("브라우저가 종료.")

else:
    print("❌ 종료할 브라우저가 없습니다.")

브라우저가 종료.
